# Generate input for negative class

In [2]:
import os
from tqdm import tqdm
import random 
import string
from perdido.geoparser import Geoparser
from utils_functions import load_json, save_json, load_edda_dataframe, run_perdido, get_term_occurrences_from_ene, segment_sentences, get_lexicon_occurrences, get_ngrams_wt_term_outside_ene

[nltk_data] Downloading package punkt to /Users/mgaio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 1. Load lexicon

The lexicon is created with the notebook `create_lexicon_from_ene.ipynb`.


In [2]:
corpus = 'Traitement_Texte'
lexicon_filename =  os.path.join('..', 'output', 'Lexicon_dataset-' + corpus + '.json')

frequency_dict_geo_tt = load_json(lexicon_filename)

In [3]:
len(frequency_dict_geo_tt)

190

# 2. Generate negative ngrams

## 2.1 Load EDdA dataset

Load the csv of EDdA dataset as a dataframe. Article from the Encyclopédie will be used to generate negative ngrams with words from the lexicon as pivot.


This repository use Git LFS to store the file `data/EDdA_dataset_articles_superdomainBERT_230327.tsv` 
 * [Git LFS install](https://docs.github.com/fr/repositories/working-with-files/managing-large-files/installing-git-large-file-storage)
 * [Git LFS configuration](https://docs.github.com/fr/repositories/working-with-files/managing-large-files/configuring-git-large-file-storage)


In [3]:
edda_dataset_path = os.path.join('..', '..','Learning_Nominal_Entity','Corpus', 'EDdA_dataset_articles_superdomainBERT_230327.tsv')
data = load_edda_dataframe(edda_dataset_path, 'Philosophie')
data.head()

,volume,numero,head,author,content,domain
0,1,1,Title Page,unsigned,"\n\nENCYCLOPÉDIE,\nDICTIONNAIRE RAISONNÉ\nDES ...",Philosophie
1,1,2,A MONSEIGNEUR LE COMTE D'ARGENSON,Diderot & d'Alembert,"\n\nA MONSEIGNEUR\nLE COMTE D'ARGENSON,\nMINIS...",Philosophie
3,1,5,"A, a & a",Dumarsais5,"\nA, a & a s.m. (ordre Encyclopéd.\nEntend. Sc...",Philosophie
4,1,6,A,Dumarsais5,"\nA, mot, est 1. la troisieme personne du prés...",Philosophie
5,1,7,A,Dumarsais,"\nA, préposition vient du latin à, à dextris, ...",Philosophie


In [4]:
data.shape

(4059, 6)

In [5]:
#df = data.head()
df = data.copy()

## 2.1 Build batches of sentences

In [6]:
#sentences_per_article = data['content'].apply(segment_sentences)
sentences_per_article = df['content'].apply(segment_sentences)

all_sentences = [sentence for sentences in sentences_per_article for sentence in sentences]

# Print the list of sentences
#for sentence in all_sentences:
#    print(sentence)

# build batches of sentences
batch_size = 50
batches = [all_sentences[i:i + batch_size] for i in range(0, len(all_sentences), batch_size)]

# turn each batch into a concatenate string
batch_strings = [' '.join(batch) for batch in batches]

In [7]:
len(batch_strings)

1785

## 2.2 Run Perdido on the batches of sentences

In [8]:
perdido_dir = os.path.join('..','output', 'Philosophie')
#print (perdido_dir)
os.makedirs(perdido_dir, exist_ok=True)

In [9]:
geoparser = Geoparser(version='Standard', sources=['wiki_gaz'])

for batch in tqdm(batch_strings):
    # generate a random string of 10 characters
    filename = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
    doc = run_perdido(batch, geoparser)
    try:
        doc.to_xml(os.path.join(perdido_dir, filename + '.xml'))
    except:
        print('Error', filename)


 10%|██████████████████                                                                                                                                                                      | 175/1785 [4:09:01<38:11:02, 85.38s/it]


KeyboardInterrupt: 

## 2.3 Get frequency of terms from the lexicon in the ENE 


* Regarde pour chaque terme du lexique (TT) sa fréquence dans les ENE des articles de Philosophie. 

In [15]:
terms = []
for doc in tqdm(sorted(os.listdir(perdido_dir))):
    terms.extend(get_term_occurrences_from_ene(os.path.join(perdido_dir,doc)))

frequency_dict = get_lexicon_occurrences(terms, frequency_dict_geo_tt.keys())

sorted_items = sorted(frequency_dict.items(), key=lambda x: x[1], reverse=True)
print(sorted_items)

100%|████████████████████████████████████████████████████████████████████████████████| 3300/3300 [00:06<00:00, 501.90it/s]

[('', 4013), ('chemin', 0), ('lieu-dit', 0), ('chemins', 0), ('plaine', 0), ('sommet de la ligne', 0), ('sommet', 0), ('sentier', 0), ('champ', 0), ('col', 0), ('rocher', 0), ('sentiers', 0), ('route', 0), ('piste', 0), ('lac', 0), ('parking', 0), ('intersection', 0), ('auberge', 0), ('port de chaussures', 0), ('port', 0), ('massif', 0), ('vallée', 0), ('crête', 0), ('sommets', 0), ('grotte', 0), ('gare', 0), ('place', 0), ('ruisseau', 0), ('parking du lac', 0), ('rochers', 0), ('source du lac', 0), ('source', 0), ('parking de la route', 0), ('ferme', 0), ('refuge', 0), ('chalet', 0), ('auberge du lac', 0), ('ruines', 0), ('parking du château', 0), ('château', 0), ('barrage', 0), ('villages', 0), ('cascades', 0), ('châteaux', 0), ('abbaye', 0), ('rue', 0), ('rivière', 0), ('carrefour', 0), ('cour', 0), ('route des châteaux', 0), ('village', 0), ('parking au pied', 0), ('galeries', 0), ("parking de l' auberge", 0), ('pistes', 0), ('versant', 0), ('passages', 0), ('sources', 0), ('moulin

## 2.4 Build ngrams


* construire les grams, fonction : get_ngrams_wt_term_outside_ene()
* taille du ngram par défaut : 7

In [16]:
ngrams = []
for file in tqdm(sorted(os.listdir(perdido_dir))):
    ngrams.extend(get_ngrams_wt_term_outside_ene(os.path.join(perdido_dir, file), frequency_dict, 1, label='0'))

  2%|█▍                                                                                | 56/3300 [00:00<00:10, 311.28it/s]

gouvernement de l' [ état ] , vous vous 
gouvernement de l' [ état ] . L' argument 
le diminutif de [ maison ] , monticule l' 
l' est de [ mont ] ou montagne ; 
de mont ou [ montagne ] ; globule est 
habitant de terre [ ferme ] & l' habitant 
lui une feconde [ source ] de motifs , 
être souvent en [ état ] de prendre le 
& d' un [ état ] futur . M. 
suivi le même [ plan ] ; tous ont 
ce sujet un [ passage ] digne d' être 
nulle part une [ ville ] sans la connoissance 
envisageoit comme une [ source ] féconde de vertus 
vais transcrire quelques [ passages ] qui confirment ce 
, homme d' [ état ] , & qui 
fournira le second [ passage ] . Ce sage 
on à donné [ lieu ] de les commettre 
aller dans les [ lieux ] où elle pouvoit 
lui en est [ commune ] avec d' autres 
selon la doctrine [ commune ] , il ne 
selon l' opinion [ commune ] l' ame humaine 
grand nombre de [ passages ] de ses livres 
lune , en [ mer ] , en arbre 
est que la [ cour ] , la mer 
cour , la [ mer ] , & ton 
marquée pour 

  5%|███▋                                                                             | 149/3300 [00:00<00:07, 409.81it/s]

Car si on [ place ] l' oeil en 
Dans un miroir [ plan ] , chaque point 
vû dans l' [ intersection ] B de la 
dans le même [ lieu ] , quel que 
miroir dans le [ lieu ] de l' image 
d' un miroir [ plan ] . Placez pour 
bien verticalement au [ sommet ] ; mesurez l' 
chacune , au [ lieu ] que la seconde 
par une évaluation [ commune ] le produit de 
inondations de la [ mer ] , les guerres 
siege de la [ ville ] de Troie , 
plus de grands [ villages ] , dont il 
a de la [ ville ] de Thèbes . 
enfans dans les [ rues ] de Paris , 
l' on a [ lieu ] de craindre un 
nom propre de [ ville ] ; & que 
de la regle [ commune ] , qui dit 
du datif au [ lieu ] de l' ablatif 
-il arrivé au [ lieu ] de son exil 
dans le déplorable [ état ] où on l' 
& au - [ lieu ] d' appeller un 
attirer à la [ cour ] de Julien . 
au milieu des [ ruines ] de la secte 
prenoit dans les [ rues ] spatieuses , marchant 
. Il étoit [ ferme ] dans ses sentimens 
dit d' un [ chemin ] qu' il est 
à la regle [ commune ] de tous 

  7%|█████▉                                                                           | 244/3300 [00:00<00:06, 445.37it/s]

...

qui indiqueroit la [ place ] de chaque mot 
mot les meilleurs [ passages ] des excellens poëtes 
quelque sorte la [ commune ] mesure de toutes 
mots , au [ lieu ] que dictionnaire en 
affaires de l' [ état ] , il s' 
amour ; au [ lieu ] que si l' 
adjectivement ; au [ lieu ] que dans le 
employé : au [ lieu ] que si l' 
singulier , la [ maison ] de mon voisin 
pluriel , les [ maisons ] d' une telle 
d' une telle [ ville ] sont bâties de 
li Vénitiens par [ mer ] , & li 
réunion de l' [ église ] anglicane avec l' 
anglicane avec l' [ église ] luthérienne . Il 
gratifia de la [ place ] de conseiller & 
pas sur le [ champ ] pour l' Allemagne 
côté de la [ cour ] impériale , & 
survenue dans cette [ ville ] rendit inutiles tous 
. Dans cet [ état ] il méditoit encore 
, gens de [ cour ] , soldats , 
, & donne [ lieu ] aux méprises . 
il a donné [ lieu ] à une méprise 
Latins bien des [ passages ] obscurs , qui 
, comprenant une [ chapelle ] & une habitation 
, avec un [ jardin ] qui fourni

 98%|██████████████████████████████████████████████████████████████████████████████▋ | 3246/3300 [00:07<00:00, 467.21it/s]

êtres est l' [ état ] opposé à leur 
fruits . Une [ ferme ] produit tant à 
produire à la [ cour ] . Les petites 
& suivre la [ route ] qu' on a 
barbares , au [ lieu ] des vérités solides 
sanctifiantes , au [ lieu ] que l' application 
tems après la [ ruine ] de Jérusalem . 
eut ruiné la [ ville ] de Jérusalem . 
, sur une [ montagne ] qui n' est 
sur la même [ montagne ] . Je rapporte 
l' autre les [ ruines ] de Jérusalem , 
années avant la [ ruine ] de Jérusalem . 
serions dans un [ état ] de stupeur parfaite 
. Or cet [ état ] est celui de 
réfléchis sont la [ source ] la plus féconde 
substance nécessaire , [ source ] & principe de 
Dieu est la [ source ] & des existences 
. Dans cet [ état ] , le verbe 
ont pris une [ route ] différente ; ils 
qu' ils tiennent [ lieu ] de l' infinitif 
. En premier [ lieu ] , ce gérondif 
. En second [ lieu ] , ce même 
connoissons pas la [ source ] d' où elles 
oseroit quitter le [ lieu ] qu' il occupe 
le plus vaste [ champ ] de nos connoissan

100%|████████████████████████████████████████████████████████████████████████████████| 3300/3300 [00:07<00:00, 438.78it/s]

personnes de la [ cour ] s' accorderont à 
fixe , au [ lieu ] qui la plus 
partie de la [ cour ] , conformément à 
partie de la [ cour ] ; c' est 
langage . La [ cour ] est dans la 
partent de la [ cour ] , comme de 
de la premiere [ source ] , pour se 
, par le [ canal ] des lois , 
adressent à la [ cour ] leurs sollicitations , 
raisonnable que la [ cour ] , protectrice de 
nombreuse de la [ cour ] l' emporte sur 
donnions à la [ cour ] , elle n' 
faut que la [ cour ] & les bons 
commerce de la [ cour ] & des parties 
langage de la [ cour ] pour lui faire 
protection ; la [ cour ] doit entendre le 
. que la [ cour ] ne contribue incomparablement 
langage de la [ cour ] , qui marque 
Allemagne ; chaque [ état ] peut prétendre à 
qu' un autre [ état ] , la regle 
parle à la [ cour ] du pape , 
du Cependant la [ cour ] du grand-duc de 
ce point la [ cour ] de Rome ; 
; au - [ lieu ] que la cour 
lieu que la [ cour ] de Rome l' 
plaire à la [ cour ] pour y réussir 
art a ses [ lieux ] . 

In [17]:
save_json(os.path.join('..', 'output', 'Ngrams_Class0_dataset-Encyclopedie_lexicon-Traitement_Texte.json'), ngrams)

* Regarde pour chaque terme du lexique (TT) sa fréquence en dehors des ENE des articles de Philosophie. 

In [18]:
pivots = [ngram['pivot'] for ngram in ngrams]
frequency_dict = get_lexicon_occurrences(pivots, frequency_dict_geo_tt.keys())

sorted_items = sorted(frequency_dict.items(), key=lambda x: x[1], reverse=True)
print(sorted_items)

[('lieu', 3646), ('état', 3157), ('place', 950), ('porte', 794), ('commune', 791), ('source', 658), ('ville', 590), ('maison', 566), ('passage', 393), ('lieux', 361), ('cour', 329), ('mer', 304), ('chemin', 293), ('plan', 231), ('col', 227), ('route', 200), ('champ', 178), ('sources', 178), ('ferme', 177), ('ruine', 177), ('maisons', 174), ('église', 159), ('places', 143), ('passages', 127), ('chemins', 116), ('pointe', 110), ('montagne', 92), ('rue', 90), ('champs', 81), ('jardin', 80), ('mont', 77), ('région', 74), ('forêts', 71), ('canal', 68), ('ruines', 67), ('domaine', 65), ('torrent', 60), ('mur', 59), ('rues', 49), ('sommet', 42), ('forêt', 40), ('rochers', 38), ('plaine', 35), ('ruisseau', 35), ('port', 33), ('village', 31), ('hôtel', 22), ('ponts', 22), ('rocher', 20), ('couvent', 18), ('fontaine', 18), ('canton', 17), ('moulin', 16), ('fossé', 16), ('abbaye', 14), ('bourg', 14), ('château', 13), ('lac', 12), ('vallée', 12), ('chapelle', 12), ('avenue', 10), ('vallon', 9), ('

## 2.5 Build ngrams of size 13

In [ ]:
ngram_size = 13
ngrams = []
for file in sorted(os.listdir(perdido_dir)):
    ngrams.extend(get_ngrams_wt_term_outside_ene(os.path.join(perdido_dir, file), frequency_dict, 1, label='0', ngram_size=13))

save_json(os.path.join('..', 'output', 'Ngrams_Class0_dataset-Encyclopedie_lexicon-Traitement_Texte_ngramSize-' + str(ngram_size) + '.json'), ngrams)